# Master Medication & Prevention Feature Merger

## Description
This script executes the final consolidation step for Phase 2 (The Medication Pipeline). It performs a series of Outer Joins to merge the individual feature files generated from the EMAR, Prescriptions, Pharmacy, and POE Detail modules. It handles data cleanup by imputing missing values with 0 (assuming null means zero events) and casting floating-point columns back to integers to ensure data consistency for the machine learning model.

## Clinical Justification for HAPI Research
* **Care Gap Identification:** By merging "Ordered" datasets (Prescriptions) with "Administered" datasets (EMAR), this script allows us to calculate derived features like `high_risk_ordered_not_admin`. A gap here (drug ordered but not given) could indicate patient refusal or clinical instability, both of which are risk factors for HAPI.
* **Holistic Risk Profile:** Combining pharmacological data (vasopressors/sedatives that affect perfusion and mobility) with physical prevention data (specialty mattresses/heel protectors from POE) creates a complete picture of the patient's extrinsic and intrinsic defense mechanisms against pressure injuries.
* **Data Integrity:** HAPI events are rare, and clinical data is messy. Rigorous type-casting and imputation at this stage prevent "noise" from confusing the XGBoost algorithm later, ensuring that a "0" is treated as a true absence of risk factor.

## Inputs & Outputs
* **Inputs:**
    * `emar_feat.csv` (Administration logs)
    * `prescriptions_feat.csv` (Provider orders)
    * `pharmacy_feat.csv` (Dispensing records)
    * `poe_detail_feat.csv` (Prevention devices like mattresses/dressings)
* **Output:** `medications_master_feat.csv`
* **Key Derived Features:** `high_risk_ordered_not_admin`, `vasoactive_ordered_not_admin`, `total_meds_orders` vs `total_meds_admin`

In [1]:
import os
import pandas as pd

In [2]:
# Configuration
BASE_DIR = r"D:\School\5141"

EMAR_FEAT_PATH         = os.path.join(BASE_DIR, "emar_feat.csv")
PRESC_FEAT_PATH        = os.path.join(BASE_DIR, "prescriptions_feat.csv")
PHARM_FEAT_PATH        = os.path.join(BASE_DIR, "pharmacy_feat.csv")
POE_DETAIL_FEAT_PATH   = os.path.join(BASE_DIR, "poe_detail_feat.csv")
OUTPUT_PATH            = os.path.join(BASE_DIR, "medications_master_feat.csv")

In [3]:
#Loader function
def load_feat(path: str, name: str):
    """
    Load a feature CSV if it exists. Ensure hadm_id is Int64.
    """
    if not os.path.exists(path):
        raise FileNotFoundError(f"{name} file not found at: {path}")

    df = pd.read_csv(path, low_memory=False)
    if "hadm_id" not in df.columns:
        raise ValueError(f"{name} is missing 'hadm_id' column.")

    df["hadm_id"] = df["hadm_id"].astype("Int64")
    return df

In [4]:
#Build master table function
def build_master_table():
    # Load all the individual feature tables
    emar  = load_feat(EMAR_FEAT_PATH,       "EMAR features")
    presc = load_feat(PRESC_FEAT_PATH,      "Prescriptions features")
    pharm = load_feat(PHARM_FEAT_PATH,      "Pharmacy features")
    poe_d = load_feat(POE_DETAIL_FEAT_PATH, "POE_DETAIL features")

    # Merge them step-by-step using outer joins on hadm_id
    # Start with EMAR, merge Prescriptions
    master = emar.merge(presc, on="hadm_id", how="outer", suffixes=("", "_presc"))
    # Merge Pharmacy
    master = master.merge(pharm, on="hadm_id", how="outer", suffixes=("", "_pharm"))
    # Merge POE Detail (Combined)
    master = master.merge(poe_d, on="hadm_id", how="outer", suffixes=("", "_poedet"))

    # Fill NA with 0 for all numeric cols (we'll re-cast important ones to Int64)
    master = master.fillna(0)

    # Get a column if present, else 0
    def get_col(df, col):
        return df[col] if col in df.columns else 0

    # Total medication "orders" (from prescriptions & pharmacy)
    master["total_meds_orders"] = (
        get_col(master, "num_prescriptions")
        + get_col(master, "num_pharm_orders")
    )

    # Total medication administrations (EMAR)
    master["total_meds_admin"] = get_col(master, "num_meds_admin")

    
    # Ordered vs administered flags (high-level)
  
    # Function to make 0/1 Int64 safely
    def to_int_flag(series):
        return series.astype("Int64") if hasattr(series, "astype") else pd.Series(dtype="Int64")

    # High-risk ordered vs given
    if "has_high_risk_drug" in master.columns and "has_high_risk_med" in master.columns:
        ordered = to_int_flag(master["has_high_risk_drug"])
        given   = to_int_flag(master["has_high_risk_med"])
        master["high_risk_ordered_not_admin"] = ((ordered == 1) & (given == 0)).astype("Int64")
    else:
        master["high_risk_ordered_not_admin"] = 0

    # Vasoactive ordered vs given
    if "has_vasoactive_drug" in master.columns and "has_vasoactive_med" in master.columns:
        ordered = to_int_flag(master["has_vasoactive_drug"])
        given   = to_int_flag(master["has_vasoactive_med"])
        master["vasoactive_ordered_not_admin"] = ((ordered == 1) & (given == 0)).astype("Int64")
    else:
        master["vasoactive_ordered_not_admin"] = 0

    # Sedation ordered vs given
    if "has_sedation_drug" in master.columns and "has_sedation_med" in master.columns:
        ordered = to_int_flag(master["has_sedation_drug"])
        given   = to_int_flag(master["has_sedation_med"])
        master["sedation_ordered_not_admin"] = ((ordered == 1) & (given == 0)).astype("Int64")
    else:
        master["sedation_ordered_not_admin"] = 0

    # Opioids ordered vs given
    if "has_opioid_drug" in master.columns and "has_opioid_med" in master.columns:
        ordered = to_int_flag(master["has_opioid_drug"])
        given   = to_int_flag(master["has_opioid_med"])
        master["opioids_ordered_not_admin"] = ((ordered == 1) & (given == 0)).astype("Int64")
    else:
        master["opioids_ordered_not_admin"] = 0

    # Re-cast important integer columns to Int64
    int_cols = [
        # From individual tables 
        "num_meds_admin",
        "num_distinct_meds",
        "num_high_risk_meds",
        "has_high_risk_med",
        "has_vasoactive_med",
        "has_sedation_med",
        "has_opioid_med",
        "num_prescriptions",
        "num_distinct_drugs",
        "num_high_risk_drugs",
        "has_high_risk_drug",
        "has_vasoactive_drug",
        "has_sedation_drug",
        "has_opioid_drug",
        "num_pharm_orders",
        "num_iv_meds",
        "has_topical_skin_med",
        "has_nutrition_support",
        "num_orders",
        "num_nursing_orders",
        "has_wound_order",
        "has_mobility_restriction",
        "num_detail_items",
        "has_specialty_mattress",
        "has_heel_protector",
        "has_foam_dressing",
        "has_wound_vac",
        "has_turning_schedule",
        "total_meds_orders",
        "total_meds_admin",
        "high_risk_ordered_not_admin",
        "vasoactive_ordered_not_admin",
        "sedation_ordered_not_admin",
        "opioids_ordered_not_admin",
    ]

    for col in int_cols:
        if col in master.columns:
            master[col] = master[col].astype("Int64")

    # Sort by hadm_id
    master = master.sort_values("hadm_id").reset_index(drop=True)

    print("Master medications + POE features built:")
    print(master.head())

    return master

In [5]:
# Execute
if __name__ == "__main__":
    master = build_master_table()
    print(f"Saving to: {OUTPUT_PATH}")
    master.to_csv(OUTPUT_PATH, index=False)
    print("Done.")

Master medications + POE features built:
    hadm_id  num_meds_admin  num_distinct_meds  num_high_risk_meds  \
0  20000019               0                  0                   0   
1  20000024              19                  8                   2   
2  20000034             129                 18                   0   
3  20000041               0                  0                   0   
4  20000045             581                 21                  32   

   has_high_risk_med  has_vasoactive_med  has_sedation_med  has_opioid_med  \
0                  0                   0                 0               0   
1                  1                   0                 0               1   
2                  0                   0                 0               0   
3                  0                   0                 0               0   
4                  1                   0                 0               1   

   num_prescriptions  num_distinct_drugs  ...  has_heel_protector  \
